In [ ]:
import os

os.environ['AWS_PROFILE'] = 'admin'
os.environ['HAVEN_DATABASE'] = 'haven'

from mirrorverse.utils import read_data_w_cache, find_neighbors

from haven.db import write_data

import h3
import plotly.express as px
from tqdm import tqdm
from collections import defaultdict
import pandas as pd
import numpy as np
import geopy.distance

In [ ]:
sql = ''' 
select 
    round(lat, 1) as lat,
    round(lon, 1) as lon,
    avg(elevation) as elevation
from 
    elevation_uploads
group by 
    1, 2
'''
data = read_data_w_cache(sql)
print(data.shape)
data.head()

In [ ]:
px.scatter_geo(
    data[data['elevation'] > 0].sample(10000), lat='lat', lon='lon',
    color='elevation',
)

In [ ]:
water = defaultdict(list)
land = []
for _, row in tqdm(data.iterrows()):
    lat, lon, elevation = row['lat'], row['lon'], row['elevation']
    if elevation < 0:
        water[lat].append(lon)
    if elevation >= 0:
        land.append((lat, lon))

In [ ]:
coastal = []
increment = 0.1
for lat, lon in tqdm(land):
    is_coastal = False
    for look_lat in [lat - increment, lat, lat + increment]:
        for look_lon in [lon - increment, lon, lon + increment]:
            if look_lon in water[look_lat]:
                is_coastal = True
                break
        if is_coastal:
            break
    if is_coastal:
        coastal.append((lat, lon))

coastal_df = pd.DataFrame(coastal, columns=['lat', 'lon'])
print(coastal_df.shape)
coastal_df.head()

In [ ]:
px.scatter_geo(
    coastal_df, lat='lat', lon='lon',
)

In [ ]:
px.scatter_geo(
    data[data['elevation'] < -2000].sample(10000), lat='lat', lon='lon',
    color='elevation',
)

In [ ]:
basin = defaultdict(list)
coast = []
for _, row in tqdm(data.iterrows()):
    lat, lon, elevation = row['lat'], row['lon'], row['elevation']
    if elevation < -2000:
        basin[lat].append(lon)
    if 0 > elevation >= -2000:
        coast.append((lat, lon))

In [ ]:
drop = []
increment = 0.1
for lat, lon in tqdm(coast):
    is_drop = False
    for look_lat in [lat - increment, lat, lat + increment]:
        for look_lon in [lon - increment, lon, lon + increment]:
            if look_lon in basin[look_lat]:
                is_drop = True
                break
        if is_drop:
            break
    if is_drop:
        drop.append((lat, lon))

drop_df = pd.DataFrame(drop, columns=['lat', 'lon'])
print(drop_df.shape)
drop_df.head()

In [ ]:
px.scatter_geo(
    drop_df, lat='lat', lon='lon',
)

In [ ]:
coastal_df['case'] = 'coastline'
drop_df['case'] = 'dropoff'
boundaries = pd.concat([coastal_df, drop_df])
px.scatter_geo(
    boundaries, lat='lat', lon='lon',
    color='case',
)

In [ ]:
neighbors = {}
allowed = {}
to_close = {}

NUM_NEIGHBORS = 5
MAX_DISTANCE = 2
MIN_DISTANCE = 0.5

points = [
    [lat, lon] for lat, lon in zip(drop_df['lat'], drop_df['lon'])
]
for lat, lon in tqdm(points):
    drop_df['distance'] = ((drop_df['lat'] - lat) ** 2 + (drop_df['lon'] - lon) ** 2) ** 0.5
    df = drop_df[drop_df['distance'] > 0].sort_values('distance', ascending=True)
    neighbors[(lat, lon)] = {(lat_n, lon_n) for (lat_n, lon_n) in df[df['distance'] <= MAX_DISTANCE][['lat', 'lon']].head(NUM_NEIGHBORS).values.tolist()}
    allowed[(lat, lon)] = {(lat_n, lon_n) for (lat_n, lon_n) in df[df['distance'] <= MAX_DISTANCE][['lat', 'lon']].values.tolist()}
    to_close[(lat, lon)] = {(lat_n, lon_n) for (lat_n, lon_n) in df[df['distance'] <= MIN_DISTANCE][['lat', 'lon']].values.tolist()}

In [ ]:
expanded_neighbors = {}
for (lat, lon), neighbor_set in tqdm(neighbors.items()):
    allowed_set = allowed[(lat, lon)]

    expanded_neighbors_set = {e for e in neighbor_set}

    while neighbor_set:
        next_level = set()
        for lat_n, lon_n in neighbor_set:
            new = neighbors[(lat_n, lon_n)]
            for lat_n2, lon_n2 in new:
                if (lat_n2, lon_n2) not in expanded_neighbors_set and (lat_n2, lon_n2) in allowed_set:
                    expanded_neighbors_set.add((lat_n2, lon_n2))
                    next_level.add((lat_n2, lon_n2))
        neighbor_set = next_level
    
    expanded_neighbors[(lat, lon)] = expanded_neighbors_set

In [ ]:
rows = []
for i, ((lat, lon), neighbor_set) in tqdm(enumerate(expanded_neighbors.items())):
    neighbor_set -= to_close[(lat, lon)]
    for lat_n, lon_n in neighbor_set:
        rows.append((i, lat, lon, lat_n, lon_n))
df = pd.DataFrame(rows, columns=['i', 'lat', 'lon', 'lat_n', 'lon_n'])
df['distance'] = ((df['lat_n'] - df['lat']) ** 2 + (df['lon_n'] - df['lon']) ** 2) ** 0.5
df['lat_diff'] = df['lat'] - df['lat_n']
df['lon_diff'] = df['lon'] - df['lon_n']
df.loc[df['lon_diff'] < 0, 'lat_diff'] = -df['lat_diff']
df.loc[df['lon_diff'] < 0, 'lon_diff'] = -df['lon_diff']
df['lat_diff'] = df['lat_diff'] / df['distance']
df['lon_diff'] = df['lon_diff'] / df['distance']
df['angle'] = np.arctan2(df['lat_diff'], df['lon_diff'])
df = df.groupby(['lat', 'lon'])['angle'].mean().reset_index()
shelf_angle = df

In [ ]:
df[['lat', 'lon']].drop_duplicates().shape

In [ ]:
px.scatter_geo(
    shelf_angle, lat='lat', lon='lon',
    color='angle',
)

In [ ]:
sql = '''
select 
    *
from 
    mean_elevation_by_h3
where 
    h3_resolution = 4
'''
h3_data = read_data_w_cache(sql)
print(h3_data.shape)
h3_data.head()

In [ ]:
if os.path.exists('doubles.csv'):
    doubles = pd.read_csv('doubles.csv')
else:
    doubles = []
    for h3_index in tqdm(list(h3_data['h3_index'])):
        neighbors = find_neighbors(100, h3_index)
        for neighbor in neighbors:
            if neighbor in h3_data['h3_index'].values:
                doubles.append((h3_index, neighbor))
    doubles = pd.DataFrame(doubles, columns=['h3_index', 'neighbor'])
    doubles.to_csv('doubles.csv', index=False)

In [ ]:
coast_match = []
df = coastal_df.copy()
for h3_index in tqdm(list(h3_data['h3_index'])):
    lat, lon = h3.h3_to_geo(h3_index)
    df['lat_h3'] = lat
    df['lon_h3'] = lon if lon > 0 else lon + 360
    df['distance'] = ((df['lat_h3'] - df['lat']) ** 2 + (df['lon_h3'] - df['lon']) ** 2) ** 0.5
    x = df.sort_values('distance', ascending=True).head(1)
    coast_match.append((h3_index, x['lat'].values[0], x['lon'].values[0], x['distance'].values[0]))
coast_match = pd.DataFrame(coast_match, columns=['h3_index', 'lat', 'lon', 'distance'])
print(coast_match.shape)
coast_match.head()

In [ ]:
shelf_match = []
df = shelf_angle.copy()
for h3_index in tqdm(list(h3_data['h3_index'])):
    lat, lon = h3.h3_to_geo(h3_index)
    df['lat_h3'] = lat
    df['lon_h3'] = lon if lon > 0 else lon + 360
    df['distance'] = ((df['lat_h3'] - df['lat']) ** 2 + (df['lon_h3'] - df['lon']) ** 2) ** 0.5
    x = df.sort_values('distance', ascending=True).head(1)
    shelf_match.append((h3_index, x['lat'].values[0], x['lon'].values[0], x['distance'].values[0]))
shelf_match = pd.DataFrame(shelf_match, columns=['h3_index', 'lat', 'lon', 'distance'])
print(shelf_match.shape)
shelf_match.head()

In [ ]:
df = shelf_match.copy()
df['lat_h3'] = df.apply(lambda x: h3.h3_to_geo(x['h3_index'])[0], axis=1)
df['lon_h3'] = df.apply(lambda x: h3.h3_to_geo(x['h3_index'])[1], axis=1)
px.scatter_geo(
    df.sample(10000), lat='lat_h3', lon='lon_h3',
    color='distance', range_color=[0, 2.0],
)

In [ ]:
shelf_match.merge(shelf_angle, on=['lat', 'lon'], how='inner')

In [ ]:
cdf = coast_match.rename(columns={'distance': 'coast_distance', 'lat': 'coast_lat', 'lon': 'coast_lon'})
sdf = (
    shelf_match
    .merge(shelf_angle, on=['lat', 'lon'], how='inner')
    .rename(columns={'distance': 'shelf_distance', 'lat': 'shelf_lat', 'lon': 'shelf_lon'})
)
merged = pd.merge(cdf, sdf, on='h3_index')
merged['lat'] = merged.apply(lambda x: h3.h3_to_geo(x['h3_index'])[0], axis=1)
merged['lon'] = merged.apply(lambda x: h3.h3_to_geo(x['h3_index'])[1], axis=1)
merged.loc[merged['lon'] < 0, 'lon'] += 360
merged.rename(columns={'angle': 'shelf_angle'}, inplace=True)


merged['to_coast_angle'] = np.arctan2(
    merged['coast_lat'] - merged['lat'],
    merged['coast_lon'] - merged['lon'],
)
merged['to_shelf_angle'] = np.arctan2(
    merged['shelf_lat'] - merged['lat'],
    merged['shelf_lon'] - merged['lon'],
)

merged['coast_distance'] = merged.apply(
    lambda r: geopy.distance.geodesic((r['lat'], r['lon']), (r['coast_lat'], r['coast_lon'])).km, axis=1
)
merged['shelf_distance'] = merged.apply(
    lambda r: geopy.distance.geodesic((r['lat'], r['lon']), (r['shelf_lat'], r['shelf_lon'])).km, axis=1
)

print(merged.shape)
merged.head()

In [ ]:
doubles['angle'] = np.arctan2(
    doubles['neighbor'].apply(lambda x: h3.h3_to_geo(x)[0]) - doubles['h3_index'].apply(lambda x: h3.h3_to_geo(x)[0]),
    doubles['neighbor'].apply(lambda x: h3.h3_to_geo(x)[1]) - doubles['h3_index'].apply(lambda x: h3.h3_to_geo(x)[1])
)
doubles['distance'] = doubles.apply(
    lambda r: geopy.distance.geodesic(
        h3.h3_to_geo(r['h3_index']),
        h3.h3_to_geo(r['neighbor'])
    ).km, axis=1
)
doubles

In [ ]:
df = doubles.merge(merged[['h3_index', 'to_coast_angle', 'to_shelf_angle', 'shelf_angle']], on='h3_index', how='inner')
df['parallel_to_shelf'] = df.apply(
    lambda r: np.abs(
        np.cos(r['shelf_angle']) * np.cos(r['angle']) + np.sin(r['shelf_angle']) * np.sin(r['angle'])
    ),
    axis=1
)
df['toward_coast'] = df.apply(
    lambda r: (
        np.cos(r['to_coast_angle']) * np.cos(r['angle']) + np.sin(r['to_coast_angle']) * np.sin(r['angle'])
    ),
    axis=1
)
df['toward_shelf'] = df.apply(
    lambda r: (
        np.cos(r['to_shelf_angle']) * np.cos(r['angle']) + np.sin(r['to_shelf_angle']) * np.sin(r['angle'])
    ),
    axis=1
)
df

In [ ]:
h3_index = df['h3_index'].sample(1).values[0]

#h3_index = '841391bffffffff'
print(h3_index)

sdf = df[df['h3_index'] == h3_index]
sdf['lat'] = sdf.apply(lambda x: h3.h3_to_geo(x['neighbor'])[0], axis=1)
sdf['lon'] = sdf.apply(lambda x: h3.h3_to_geo(x['neighbor'])[1], axis=1)
px.scatter_geo(
    sdf, lat='lat', lon='lon',
    color='toward_shelf'
)

In [ ]:
shelf_match[shelf_match['h3_index'] == h3_index]

In [ ]:
df['resolution'] = 4

write_data(
    df, 'options_w_angles', ['resolution']
)

In [ ]:
merged['resolution'] = 4

write_data(
    merged, 'coastal_info', ['resolution']
)